In [ ]:
import random
import numpy as np
import pygame
pygame.init()

In [3]:
WHITE = (255, 255, 255)
RED = (200,0,0)
BLUE1 = (0, 0, 255)
BLUE2 = (0, 100, 255)
BLACK = (0,0,0)

BLOCK_SIZE = 20
SPEED = 10

In [3]:


WHITE = (255, 255, 255)
RED = (200,0,0)
BLUE1 = (0, 0, 255)
BLUE2 = (0, 100, 255)
BLACK = (0,0,0)

BLOCK_SIZE = 20
SPEED = 10

In [4]:
class SnakeGame():

    def __init__(self,player=None,grid_size=100):
        self.grid_size = grid_size
        self.head = (self.grid_size//2, self.grid_size//2)
        self.snake_body = [(self.grid_size//2, self.grid_size//2)]
        self.food = None
        self.place_food()
        self.score = 0
        pygame.init()
        #self.font = pygame.font.SysFont('arial', 25)
        self.dir = random.randint(0,3)

    def place_food(self):
        while True:
            food = (np.random.randint(0, self.grid_size), np.random.randint(0, self.grid_size))
            if food not in self.snake_body:
                self.food = food
                break

    def is_full(self):
        return len(self.snake_body)==self.grid_size**2
    
    def evaluate(self):
        self.snake_body.insert(0, self.head)
        if (
            self.head[0] < 0
            or self.head[0] >= self.grid_size
            or self.head[1] < 0
            or self.head[1] >= self.grid_size
            or self.head in self.snake_body[1:]
        ):
            return -10,True
        
        if self.head == self.food:
            self.score += 1
            self.place_food()
            return 20, False
        else:
            self.snake_body.pop()
            return 0.01, False
        
    def move(self, action):
        if action == 0:  # Up
            self.head = (self.head[0], self.head[1]-1)
        elif action == 1:  # Down
            self.head = (self.head[0]-1, self.head[1])
        elif action == 2:  # Left
            self.head = (self.head[0] , self.head[1]+1)
        elif action == 3:  # Right
            self.head = (self.head[0]+1, self.head[1])
        
    def train(self,player):
        done = False
        while not done and len(player.state_action_reward)<10000:

            state = player.get_state(self)
            next_move = player.find_move(state)
            self.move(next_move)
            self.dir = next_move
            reward,done = self.evaluate()
            player.update_sa(state,next_move,reward)
        self.reset()

    def play_self(self,player):

        self.display = pygame.display.set_mode((self.grid_size*BLOCK_SIZE, self.grid_size*BLOCK_SIZE))
        pygame.display.set_caption('Snake')
        self.clock = pygame.time.Clock()

        done = False
        while not done:
            state = player.get_state(self)
            action = player.target_policy(state)
            self.move(action)
            rew,done = self.evaluate()
            self.update_ui()
            self.clock.tick(SPEED)
        ans = self.score
        self.reset()
        pygame.quit()
        return ans


    def reset(self):
        self.head = (self.grid_size // 2, self.grid_size // 2)
        self.snake_body = [(self.grid_size // 2, self.grid_size // 2)]
        self.place_food()
        self.score = 0
    
    def update_ui(self):
        self.display.fill(BLACK)
        
        pygame.draw.rect(self.display,RED, pygame.Rect(self.head[0]*BLOCK_SIZE -2,self.head[1]*BLOCK_SIZE - 2,24,24))
        for pt in self.snake_body:
            pygame.draw.rect(self.display, BLUE1, pygame.Rect(pt[0]*BLOCK_SIZE, pt[1]*BLOCK_SIZE, BLOCK_SIZE, BLOCK_SIZE))
            pygame.draw.rect(self.display, BLUE2, pygame.Rect(pt[0]*BLOCK_SIZE+4, pt[1]*BLOCK_SIZE+4, 12, 12))
            
        pygame.draw.rect(self.display, RED, pygame.Rect(self.food[0]*BLOCK_SIZE, self.food[1]*BLOCK_SIZE, BLOCK_SIZE, BLOCK_SIZE))
        
        #text = font.render("Score: " + str(self.score), True, WHITE)
        #self.display.blit(text, [0, 0])
        pygame.display.flip()


In [5]:
class SnakeEngine():

    def __init__(self,eps=1.1,grid_size=100,gamma = 0.9):
        self.grid_size = grid_size
        self.eps = eps #epsilon
        self.state_action_reward = [] #stores states of an episode
        self.Q = {} #stores rewards of each episode
        self.count={} #occurence of each state
        self.policy = {} #weights of e-greedy behavioural policy
        self.gamma = gamma
        


    def get_state(self, game):
        '''
        update state here
        '''
        arr = [0]*9
        if game.head[0] - 1 < 0 or (game.head[0] - 1, game.head[1]) in game.snake_body:
            arr[0] = 1
        if game.head[1] - 1 < 0 or (game.head[0], game.head[1] - 1) in game.snake_body:
            arr[1] = 1
        if game.head[0] + 1 >= game.grid_size or (game.head[0] + 1, game.head[1]) in game.snake_body:
            arr[2] = 1
        if game.head[1] + 1 >= game.grid_size or (game.head[0], game.head[1] + 1) in game.snake_body:
            arr[3] = 1

        if game.food[0] - game.head[0] > 0:
            arr[4] = 1
        if game.food[1] - game.head[1] > 0:
            arr[5] = 1
        if -game.food[0] + game.head[0] > 0:
            arr[6] = 1
        if -game.food[1] + game.head[1] > 0:
            arr[7] = 1
        arr[8] = game.dir

        return tuple(arr)
    
    def find_move(self,state):
        if state not in self.policy:
            self.policy[state] = [self.eps/4]*4
            self.Q[state] = np.random.normal(size=4).tolist()
        if state not in self.count.keys():
            self.count[state] = [0]*4
        '''
        update b-policy here (1)
        '''
        next_move = random.choices([0,1,2,3], weights=self.policy[state], k=1)[0]  

        return next_move
    
    def update_sa(self,state,action,reward):
        self.state_action_reward.append((state, action,reward))
    
    def update_qval(self):
        returns = 0
        wis = 1

        while len(self.state_action_reward) >0 :
            state,action,reward = self.state_action_reward.pop()
            returns = self.gamma*returns + reward
            self.count[state][action] +=wis
            self.Q[state][action] += wis*(returns - self.Q[state][action])/self.count[state][action]
            wis*= sum(self.policy[state])/(self.policy[state][action])

        '''
        update b-policy here (2)
        '''
        for state in self.Q.keys():
            min_index = self.Q[state].index(min(self.Q[state]))
            self.policy[state] = [self.eps / 4]*4
            self.policy[state][min_index] = 1-3*self.eps/4

        self.state_action_reward = []
        self.count = {}
    
    def target_policy(self,state):
        if state in self.Q.keys():
            return self.Q[state].index(max(self.Q[state]))
        else:
            return random.randint(0,3)
        
    def complete_reset(self):
        #init conditions
        self.state_action_reward = [] 
        self.Q = {}
        self.count={} 
        self.policy = {} 


In [6]:
player = SnakeEngine()
game = SnakeGame(player,10)

In [7]:
player.complete_reset()

In [7]:
for _ in range(100000):
    game.train(player)
    player.update_qval()


In [8]:
print(len(player.Q))

239


In [10]:
game.play_self(player)

: 

: 